In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("KODEX200_monthly_data.csv", encoding="utf-8")

In [3]:
df.head()

,Date,Open,High,Low,Close,Volume,Change
0,2010-01-29,19052,19066,18668,18742,2717811,-0.026996
1,2010-02-26,18611,18683,18514,18559,974615,0.000431
2,2010-03-31,19853,19858,19729,19789,883019,-0.003776
3,2010-04-30,20331,20403,20313,20377,758179,0.007814
4,2010-05-31,18887,19126,18882,19134,1979678,0.012220


In [4]:
kodex200 = df[["Date","Close"]]
kodex200.index = kodex200["Date"]
del kodex200["Date"]

### 12개월 평균 모멘텀 및 스코어 구하기

In [5]:
# 평균모멘텀 구하기
def 평균모멘텀구하기(data,month):
    result = 0
    for i in range(1,(month+1),1):
        result = data / data.shift(i) + result
    return result/month

# 평균모멘텀 스코어 구하기
def 평균모멘텀스코어구하기(data,month):
    result = 0
    for i in range(1,(month+1),1):
        result = np.where(data / data.shift(i) > 1,1,0) + result
    score = pd.DataFrame(result/month, index=data.index,columns=["score"]) 
    return score

### 12개월 평균 모멘텀 구하기

In [6]:
# 현금투자할 상수 = 현금비율 0.5는 50%를 뜻함.
cash = 0.5
# 평균모멘텀스코어구하기
score12 = 평균모멘텀스코어구하기(kodex200["Close"],12)
# 평균모멘텀스코어와 종목["Close"] 합침
momentum12 = pd.concat([kodex200, score12], axis=1)

### 주식, 현금 투자비율

In [7]:
#주식투자 비율 = 주식 평균모멘텀 스코어 / (주식 평균모멘텀 스코어 + 현금상수)
stock_rate = momentum12["score"] / (cash + momentum12["score"])
#현금보유 비율 = 1 - 주식비율
cash_rate = 1 - stock_rate

### 전략 수익곡선 6개월 평가

In [9]:
m12 = 평균모멘텀구하기(kodex200["Close"],12)
score6 = 평균모멘텀스코어구하기(m12,6)
momentum6 = pd.concat([kodex200, score6], axis=1)

### 그래프 그리기

In [ ]:
plt.title("KODEX200 Dual Momentum")
plt.plot(momentum12["score"])
plt.plot(momentum6["score"])

In [ ]:
plt.title("KODEX200 수익곡선")
plt.plot(momentum6["score"])